## Part 1 - Model the Detroit Police Population
#### Read in data from Detroit Police Reports file

In [1]:
import requests
import csv
import string

path = 'https://opendata.arcgis.com/api/v3/datasets/4f49eb825f564efa9a23cd103c4ba13b_0/downloads/data?format=csv&spatialRefId=4326'
data = requests.get(path)

lines = data.text.splitlines()

reader = csv.reader(lines)

#### Convert to List of Dictionaries

In [2]:
columns = next(reader)

new_columns = ["".join(_ for _ in name if _ in string.ascii_letters + "_").lower() for name in columns]

master_list = []
for i,row in enumerate(reader):
    row_values = [_ for _ in row]
    new_dict = dict(zip(new_columns,row_values))
    master_list.append(new_dict)
    if i > 200000:
        break

#### Filter List by Parameters

In [3]:
print("{:,}".format(len(master_list)), 'total items')
filtered_list = list(filter(lambda x: (x['zip_code'].strip() != "" and x['neighborhood'].strip() != ""), master_list))
print("{:,}".format(len(filtered_list)), 'filtered items')

200,002 total items
179,014 filtered items


#### Example Output

In [4]:
filtered_list[0]

{'x': '-82.986961795',
 'y': '42.4083576620001',
 'incident_id': '201626400002',
 'agency': 'DPD ',
 'incident_address': 'Promenade Ave & Roseberry St',
 'zip_code': '48213',
 'priority': '3',
 'callcode': '825030',
 'calldescription': 'DISTURBANCE',
 'category': 'DISTURB ',
 'call_timestamp': '2016/09/20 09:36:12+00',
 'precinct_sca': '910A',
 'respondingunit': '',
 'officerinitiated': 'No',
 'intaketime': '2.2',
 'dispatchtime': '528.6',
 'traveltime': '15.5',
 'totalresponsetime': '546.6',
 'time_on_scene': '120.5',
 'totaltime': '667.1',
 'neighborhood': 'Wade',
 'block_id': '261635043003006',
 'council_district': '4',
 'longitude': '-82.9869617954524',
 'latitude': '42.4083576622369',
 'oid': '1'}

****
#### Calculate Using Lambda and Reduce
 - Average Total Response Time
 - Average Dispatch Time
 - Average Total Time

In [5]:
from functools import reduce

average_total_response_time = reduce(
    lambda x1, x2: x1 + (
        float(x2['totalresponsetime'].replace(",","").strip()
             ) if x2['totalresponsetime'] != "" else 0
    ), filtered_list, 0
)

average_dispatch_time = reduce(
    lambda x1, x2: x1 + (
        float(x2['dispatchtime'].replace(",","").strip()
             ) if x2['dispatchtime'] != "" else 0
    ), filtered_list, 0
)

average_total_time = reduce(
    lambda x1, x2: x1 + (
        float(x2['totaltime'].replace(",","").strip()
             ) if x2['totaltime'] != "" else 0
    ), filtered_list, 0
)

avg_response_time = average_total_response_time/len(filtered_list)
avg_dispatch_time = average_dispatch_time/len(filtered_list)
avg_total_time = average_total_time/len(filtered_list)

print(f'The average total response time is {avg_response_time} minutes')
print(f'The average dispatch time is {avg_dispatch_time} minutes')
print(f'The average total time is {avg_total_time} minutes')

The average total response time is 32.19354519758174 minutes
The average dispatch time is 26.771945210988875 minutes
The average total time is 69.96388103723928 minutes


## Part 2 - Model the Neighborhood Samples
#### Divide list of dictionaries into smaller lists of dictionaries by neighborhood


In [6]:
import collections

# The list parameter in this call is telling the function to expect a list
result = collections.defaultdict(list)

for row in filtered_list:
    result[row['neighborhood']].append(row)
    
neighborhood_separated_list = list(result.values())
print(len(neighborhood_separated_list))

207


#### Aggregate data for each neighborhood
 - Average Total Response Time
 - Average Dispatch Time
 - Average Total Time

In [7]:
end_keys = {
    'totalresponsetime': 'Average Total Response Time',
    'dispatchtime': 'Average Dispatch Time',
    'totaltime': 'Average Total Time'}

neighborhood_data = []
for neighborhood in neighborhood_separated_list:
    neighborhood_result = {}
    for target in end_keys.keys():
        new_key = end_keys[target]
        target_val = reduce(lambda x1, x2: x1 + (float(x2[target].replace(",","")) if x2[target] != '' else 0), neighborhood, 0)
        target_avg = target_val/len(neighborhood)
        neighborhood_result[new_key] = target_avg
    neighborhood_dict = {neighborhood[0]['neighborhood']: neighborhood_result}
    neighborhood_data.append(neighborhood_dict)
    
neighborhood_data[:4]    

[{'Wade': {'Average Total Response Time': 44.901331114808656,
   'Average Dispatch Time': 39.533111480865216,
   'Average Total Time': 85.58569051580695}},
 {'Tri-Point': {'Average Total Response Time': 20.570000000000004,
   'Average Dispatch Time': 14.546363636363633,
   'Average Total Time': 51.71636363636361}},
 {'Oak Grove': {'Average Total Response Time': 32.408713692946066,
   'Average Dispatch Time': 26.978008298755164,
   'Average Total Time': 67.89142461964032}},
 {'We Care Community': {'Average Total Response Time': 33.69770270270268,
   'Average Dispatch Time': 27.86405405405405,
   'Average Total Time': 65.2475675675675}}]

#### Add a dictionary item to include population of Detroit in combined list

In [8]:
neighborhood_data.insert(0,{"Population of Detroit": 674841})
neighborhood_data[:4]

[{'Population of Detroit': 674841},
 {'Wade': {'Average Total Response Time': 44.901331114808656,
   'Average Dispatch Time': 39.533111480865216,
   'Average Total Time': 85.58569051580695}},
 {'Tri-Point': {'Average Total Response Time': 20.570000000000004,
   'Average Dispatch Time': 14.546363636363633,
   'Average Total Time': 51.71636363636361}},
 {'Oak Grove': {'Average Total Response Time': 32.408713692946066,
   'Average Dispatch Time': 26.978008298755164,
   'Average Total Time': 67.89142461964032}}]

## Part 3 - Create an Output JSON file

In [9]:
import json

neighborhood_json_file = json.dumps(neighborhood_data)

with open('detroit_neighborhood_data.json', 'w') as file:
    file.write(neighborhood_json_file)
    file.close()